# Wide_Deep instructions
Wide_Deep class helps the user to build a wide and deep neuron network for their contextual bandit algorithm, which is useful in the recommendation/personalization domain.
The class allow the user to easily build a model with multiple embeddings in both the wide and deep part. And also allow the user to build a wide-only or deep-only model.


In [14]:
import torch
import torch.nn as nn
import numpy as np
from Wide_and_Deep_model import Wide_Deep

Here is how to build a wide and deep model with multiple embeddings in both parts.

For each element in the list `['feature_1', 100, 64]`:
- `'feature_1'` is the name of the feature that will be feed into  the `'wide'` part of the model. this is just for the user's tracking purpose on the data.
- `100` is the num_embedding of that feature, for example how many unique user in the dataset.
- `64` is the embedding dimension this feature.

In [15]:

embeddings={'wide':[['feature_1', 100, 64],
                    ['feature_2', 200, 64]],
            'deep':[['feature_3', 300, 64],
                    ['feature_4', 400, 64],
                    ['feature_5', 500, 64]]
            }

wide_dim = 10
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings, deep_neurons=[32, 16], activation=nn.ReLU())
testmodel

This is a wide and deep model.


Wide_Deep(
  (wide): Module(
    (feature_1): Embedding(100, 64)
    (feature_2): Embedding(200, 64)
  )
  (deep): Module(
    (feature_3): Embedding(300, 64)
    (feature_4): Embedding(400, 64)
    (feature_5): Embedding(500, 64)
    (NN): Sequential(
      (fc0): Linear(in_features=201, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=152, out_features=3, bias=True)
)

Here is how to build a wide-only model with embeddings, the user only need to set the `deep_dim=0`

In [16]:
embeddings={'wide':[['feature_1', 100, 64],
                    ['feature_2', 200, 64]]
            }

wide_dim = 10
deep_dim = 0
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a wide-only model.


Wide_Deep(
  (wide): Module(
    (feature_1): Embedding(100, 64)
    (feature_2): Embedding(200, 64)
  )
  (deep): Module()
  (lastlayer): Linear(in_features=136, out_features=3, bias=True)
)

Here is how to build a deep-only model with embeddings. the user only need to set the `wide_dim=0` and make sure the `deep_neurons` is not empty.

In [17]:
embeddings={'deep':[['feature_3', 300, 64],
                    ['feature_4', 400, 64],
                    ['feature_5', 500, 64]]
            }

wide_dim = 0
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings, deep_neurons=[32, 16], activation=nn.ReLU())
testmodel

This is a deep-only model.


Wide_Deep(
  (wide): Module()
  (deep): Module(
    (feature_3): Embedding(300, 64)
    (feature_4): Embedding(400, 64)
    (feature_5): Embedding(500, 64)
    (NN): Sequential(
      (fc0): Linear(in_features=201, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=16, out_features=3, bias=True)
)

Here is how to build a wide and deep model without embeddings.

In [18]:
embeddings = {}

wide_dim = 10
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings, deep_neurons=[32, 16], activation=nn.ReLU())
testmodel

This is a wide and deep model.


Wide_Deep(
  (wide): Module()
  (deep): Module(
    (NN): Sequential(
      (fc0): Linear(in_features=12, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=26, out_features=3, bias=True)
)

**Note:**
1. `wide_dim` and `deep_dim` should NOT be `0` at the same time, other wise the model will raise an error.
2. Number of `embeddings` of each part (wide/deep) should not larger than the input dimension (`wide_dim`/`deep_dim`) of that part, otherwise the model will raise an error.
3. if `deep_dim` > 0, `deep_neurons` should not be empty.


## An example to test the model

In [19]:
embeddings={'wide':[['feature_1', 100, 64],
                    ['feature_2', 200, 64]],
            'deep':[['feature_3', 300, 64],
                    ['feature_4', 400, 64],
                    ['feature_5', 500, 64]]
            }

wide_dim = 10
deep_dim = 12
action_dim = 3
testmodel = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings)
testmodel

This is a wide and deep model.


Wide_Deep(
  (wide): Module(
    (feature_1): Embedding(100, 64)
    (feature_2): Embedding(200, 64)
  )
  (deep): Module(
    (feature_3): Embedding(300, 64)
    (feature_4): Embedding(400, 64)
    (feature_5): Embedding(500, 64)
    (NN): Sequential(
      (fc0): Linear(in_features=201, out_features=32, bias=True)
      (activ0): ReLU()
      (fc1): Linear(in_features=32, out_features=16, bias=True)
      (activ1): ReLU()
    )
  )
  (lastlayer): Linear(in_features=152, out_features=3, bias=True)
)

In [20]:
# A simple test on the get_z and forward functions.
# get_z function is to extract the representation which is the input of the last layer of wide and deep model. The representation is useful in algorithms like LinUCB, Thompson Sampling, etc.

batch_size = 16
embed_inputs = {}
embed_inputs['feature_1'] = torch.tensor(np.random.randint(100, size=(batch_size, 1))).float()
embed_inputs['feature_2'] = torch.tensor(np.random.randint(200, size=(batch_size, 1))).float()
embed_inputs['feature_3'] = torch.tensor(np.random.randint(300, size=(batch_size, 1))).float()
embed_inputs['feature_4'] = torch.tensor(np.random.randint(400, size=(batch_size, 1))).float()
embed_inputs['feature_5'] = torch.tensor(np.random.randint(500, size=(batch_size, 1))).float()

wide_others = torch.tensor(np.random.random(size=(batch_size, wide_dim-2))).float()
deep_others = torch.tensor(np.random.random(size=(batch_size, deep_dim-3))).float()
wide_inputs = torch.cat((embed_inputs['feature_1'], embed_inputs['feature_2'], wide_others), dim=1)
deep_inputs = torch.cat((embed_inputs['feature_3'], embed_inputs['feature_4'], embed_inputs['feature_5'], deep_others), dim=1)

inputs = torch.cat((wide_inputs, deep_inputs), dim=1)
#
zs = testmodel.get_z(inputs)
print(zs.shape)
outs = testmodel(inputs)
print(outs.shape)

torch.Size([16, 152])
torch.Size([16, 3])


If you are interested in how to use this wide and deep model in ContextualBandit algorithm, please go to [demo_Wide_and_Deep_Contextual_Bandits.ipynb](https://github.com/fellowship/space-bandits/blob/dev_tfl/Wide_and_Deep_Contextual_Bandits/demo_Wide_and_Deep_Contextual_Bandits.ipynb)